In [1]:
!pip install gradio langchain langchain_community langchain_milvus transformers pymilvus


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.3/248.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [2]:
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.0 MB/s eta 0:00:00


In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import os
import gradio as gr
import tempfile
from langchain_milvus import Milvus
from langchain_community.llms import Replicate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.embeddings import HuggingFaceEmbeddings

# Replicate token and model setup (use your token here)
os.environ['REPLICATE_API_TOKEN'] = ""
model_path = "ibm-granite/granite-3.3-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = Replicate(model=model_path, replicate_api_token=os.environ['REPLICATE_API_TOKEN'])

# Temporary Milvus DB for vector storage
db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": db_file},
    auto_id=True,
    index_params={"index_type": "AUTOINDEX"},
)

# Write your chemistry sample document to a file for loading
filename = "/content/chemistry_acids_bases.txt"
with open(filename, "w") as f:
    f.write("""
Acids and Bases Chemistry Lab Manual

Theory:
- Acids donate H⁺ (hydrogen ions); bases donate OH⁻ (hydroxide ions).
- Acid-base titration is used to determine the concentration of solutions.

Objective:
- To determine the concentration of acid or base using titration.

Apparatus:
- Burette, pipette, conical flask, beakers, indicators (phenolphthalein, methyl orange), acid and base solutions.

Procedure:
1. Rinse the burette with standard base. Fill it.
2. Pipette a measured amount of acid into a conical flask. Add indicator.
3. Titrate the acid by adding base slowly, swirling, until endpoint color change.
4. Record the burette reading.
5. Repeat for better accuracy.

Observations:
- Note initial and final burette readings.
- Volume of base used = Final - Initial readings.
- Record color change.

Calculations:
- Use normality equation: N1V1 = N2V2.
- Substitute known values to find unknown strength.

Precautions:
- Read burette at eye level.
- Add base dropwise near endpoint.
- Use white tile to observe color change.

Sample Viva Q&A:
Q: What is a neutralization reaction? A: Acid and base react to form salt and water.
Q: Why is phenolphthalein pink in base? A: It is colorless in acid, turns pink in base.
""")

# Load & split content using your tokenizer & splitter
loader = TextLoader(filename)
documents = loader.load()
splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=tokenizer.model_max_length // 2,
    chunk_overlap=0,
)
texts = splitter.split_documents(documents)

# Add doc_id metadata for traceability in Milvus
for i, doc in enumerate(texts):
    doc.metadata["doc_id"] = i + 1

# Add split documents to Milvus vector store
vector_db.add_documents(texts)

# New prompt tailored for Chemistry content with simple, clear explanations
template = """
You are a helpful Chemistry Lab Assistant specialized in acids and bases.
User Question: {question}
Provide a clear, simple explanation suitable for students.
Include safety tips and procedural guidance when relevant.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# Setup chains with your Replicate LLM and prompt
llm_chain = LLMChain(llm=model, prompt=prompt)
combine_chain = StuffDocumentsChain(llm_chain=llm_chain)

rag_chain = RetrievalQA(
    retriever=vector_db.as_retriever(),
    combine_documents_chain=combine_chain,
    return_source_documents=False
)

# Gradio interface function to ask questions
def ask_chemistry_agent(query):
    if not query.strip():
        return "Please enter a question about acids and bases."
    try:
        response = rag_chain.run(query)
        return response
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Define Gradio interface adapted for Chemistry questions
iface = gr.Interface(
    fn=ask_chemistry_agent,
    inputs=gr.Textbox(label="Ask a question about Acids and Bases Chemistry", placeholder="e.g. What is a neutralization reaction?"),
    outputs=gr.Textbox(label="Answer"),
    title="AI Agent for Acids and Bases Chemistry",
    description="Ask questions related to acids and bases lab manual, procedures, theory, and safety tips.",
    theme="default"
)

# Launch interface with sharing enabled
iface.launch(share=True)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/tmp/ipython-input-3357730153.py:100: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=prompt)
/tmp/ipython-input-3357730153.py:101: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_chain = StuffDocumentsChain(llm_chain=llm_chain)
/tmp/ipython-input-3357730153.py:103: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://p

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f13f8b549f64dd1eb4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
